## Setup

In [1]:
import cv2
import os
import shutil
import glob
import numpy as np
import torch

from google.colab.patches import cv2_imshow

In [2]:
%cd coco_kpts
!unzip coco_kpts/coco2017labels-keypoints.zip
%cd images
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip

!unzip train2017.zip
!unzip val2017.zip
%cd ../..

d:\HocDaiHoc\TLCN_human_action_recognition\TraningModel\coco_kpts
d:\HocDaiHoc\TLCN_human_action_recognition\TraningModel\coco_kpts\images


## Check dataset

In [1]:
path_anno = '/coco_kpts/labels/val2017/000000006471.txt'
path_img = path_anno.replace('labels', 'images').replace('txt', 'jpg')
img = cv2.imread(path_img)


In [ ]:
def take_point(path_anno):
    with open(path_anno, 'r') as f:
        lines = f.readlines()
    points = []
    class_ids = []
    bbox = []
    for line in lines:
        line = line.split(' ')
        class_id = int(line[0])
        for i in range(2, len(line), 3):
            if float(line[i+2]) > 0.9:
              point = [float(line[i]), float(line[i+1])]
              points.append(point)
              class_ids.append(class_id)
        x1,y1 = float(line[1])-float(line[3])/2, float(line[2])-float(line[4])/2
        x2,y2 = float(line[1])+float(line[3])/2, float(line[2])+float(line[4])/2
        bbox.append([x1,y1,x2,y2])
        # break
    return points , class_ids, bbox

In [ ]:
list_anno_path = glob.glob("/coco_kpts/labels/val2017/*.txt")
for path_anno in list_anno_path[:10]:
  path_img = path_anno.replace('labels', 'images').replace('txt', 'jpg')
  img = cv2.imread(path_img)
  h, w, c = img.shape
  print(h,w)
  points, class_ids,bboxs = take_point(path_anno)
  print(class_ids)
  for i in range(len(points)):
      cv2.circle(img, (int(points[i][0]*w), int(points[i][1]*h)), 5, (0, 0, 255), -1)

  for bbox in bboxs:
      cv2.rectangle(img, (int(bbox[0]*w), int(bbox[1]*h)), (int(bbox[2]*w), int(bbox[3]*h)), (0, 255, 0), 1)

  cv2_imshow(img)

In [ ]:
%cd yolov7
!pip install -r requirements.txt

## Train

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!python train.py --data data/coco_kpts.yaml --cfg cfg/yolov7-w6-pose.yaml --weights weights/yolov7-w6-person.pt --batch-size 4 --device $device --name yolov7-w6-pose --hyp data/hyp.pose.yaml --kpt-label --epochs 1

## Test

In [ ]:
!python test.py --data data/coco_kpts.yaml --conf 0.001 --iou 0.65 --weights runs/train/yolov7-w6-pose15/weights/best.pt --kpt-label

In [ ]:
!python test.py --data data/coco_kpts.yaml --conf 0.001 --iou 0.65 --weights /content/yolov7-w6-pose.pt --kpt-label